In [3]:
import pandas as pd
import nltk
# nltk.download('punkt')
from nltk.corpus import stopwords
from string import punctuation
import re
from nltk import probability
import random
import sklearn

Verificaremos quais palavras estão mais frequentes após alguns preprocessamentos com o nltk. Usaremos a frquencia das top 5 palavras de cada grupo como features, além da densidade gramatical

In [4]:
corpus = pd.read_csv(r"data\corpus.csv", encoding= 'utf8')

In [5]:
df10 = corpus[corpus["faixa_etaria"] =="10"]
df12 = corpus[corpus["faixa_etaria"] =="12"]
df14 = corpus[corpus["faixa_etaria"] =="14"]
df16 = corpus[corpus["faixa_etaria"] =="16"]
df18 = corpus[corpus["faixa_etaria"] =="18"]
dfL = corpus[corpus["faixa_etaria"] =="livre"]

In [6]:
df10 = df10.append(dfL)
df10["faixa_etaria"] = "10"
df16 = df16.append(df18)
df16["faixa_etaria"] = "16"

In [7]:
dfCorpus = df10.append(df12)
dfCorpus = dfCorpus.append(df14)
dfCorpus = dfCorpus.append(df16)

In [8]:
dfCorpus = dfCorpus.sample(frac=1).reset_index(drop=True)

In [9]:
dfCorpus.head()

,titulo,faixa_etaria,sinopse
0,Querido Menino,14,David Sheff (Steve Carell) é um conceituado jo...
1,Correndo Atrás de um Pai,14,Quando descobrem que a mãe (Glenn Close) sempr...
2,O Que Te Faz Mais Forte,14,As memórias de Jeff Bauman (Jake Gyllenhaal) s...
3,A Odisseia,10,O aventureiro oceânico e cineasta francês Jacq...
4,Limites,16,Laura (Vera Farmiga) é uma mulher que busca vi...


In [ ]:
corpusTrain = pd.DataFrame()

Excluindo targets vazios

In [ ]:
dfCorpus["faixa_etaria"].value_counts()

Separando o corpus por target para realizar as análises de palavras

In [ ]:
corpus10 = dfCorpus[dfCorpus["faixa_etaria"] =="10"]
corpus12 = dfCorpus[dfCorpus["faixa_etaria"] =="12"]
corpus14 = dfCorpus[dfCorpus["faixa_etaria"] =="14"]
corpus16 = dfCorpus[dfCorpus["faixa_etaria"] =="16"]

Criando stopwords com auxilio da  biblioteca nltk e stopwords públicos disponiveis em português no Github.

In [ ]:
b = pd.read_csv("data\stopwords.txt",header = None)
b = b[0].tolist()
b = [ b[i][:-1].strip() for i in range(0,len(b)) ]
stop_words = set(stopwords.words('portuguese') + list(punctuation) + b)

Chamando função que utiliza a Função FreqDist do NLTK

In [ ]:
Freq10,df10 = get_freq(corpus10)
Freq12,df12 = get_freq(corpus12)
Freq14,df14 = get_freq(corpus14)
Freq16,df16 = get_freq(corpus16)
# Freq18,df18 = get_freq(corpus18)
# FreqL,dfL = get_freq(corpusL)

Analisando as 5 principais palavras de cada classe observamos que o radical "vid" aparece em todas as categorias, sendo assim, ele foi tirado de todo o corpus por não apresentar relevancia estatistica.

In [ ]:
common10 = commom(Freq10)
common12 = commom(Freq12)
common14 = commom(Freq14)
common16 = commom(Freq16)
# common18 = commom(Freq18)
# commonL = commom(FreqL)

Calculando a densidade lexical de cada resenha e armazenado isso em uma variavel

In [ ]:
dfGram = [(len(dfCorpus.iloc[i]['sinopse'])/len(set(dfCorpus.iloc[0]['sinopse']))) for i in range(0,len(dfCorpus))]

In [ ]:
corpusTrain = corpusTrain.append(dfGram)
corpusTrain.columns = ["densidade_gramatical"]

In [ ]:
corpusTrain.head()

In [ ]:
dfGram_title = [(len(dfCorpus.iloc[i]['titulo'])/len(set(dfCorpus.iloc[0]['titulo']))) for i in range(0,len(dfCorpus))]

In [ ]:
corpusTrain["densidade_gramatical_t"] = dfGram_title

In [ ]:
score_freq = []
for filme in range(0,len(dfCorpus)):
    sinopse = dfCorpus.iloc[filme]['sinopse']
    s = generate_label(sinopse,common10,common12,common14,common16)
    score_freq.append(s)

In [ ]:
corpusTrain["score"] = score_freq

In [ ]:
corpusTrain["target"] = dfCorpus["faixa_etaria"]

In [ ]:
corpusTrain = corpusTrain[corpusTrain["target"] !="vazio"]

In [ ]:
corpusTrain = corpusTrain.dropna()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(corpusTrain[['densidade_gramatical',"score",'densidade_gramatical_t']], corpusTrain["target"], test_size=0.2,random_state=109)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
model = clf.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred

In [ ]:
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
def commom(freq):
    freq = freq.most_common(5)
    freq = [freq[i][0] for i in range(0,len(freq))]
    return freq

In [ ]:
def get_freq(corp):
    df = corp["sinopse"].tolist()
    df = "".join(df)
    df = re.findall(r"[\w']+", df)
    df = [ df[i].lower() for i in range(0,len(df)) ]
    df = [word for word in df if word not in stop_words]
    st = nltk.stem.RSLPStemmer()
    df = [ st.stem(df[i]) for i in range(0,len(df)) ]
    df = [x for x in df if x != "vid"]
    return probability.FreqDist(df), df

In [ ]:
def generate_label(sinopse,c10,c12,c14,c16):

    a = sinopse
    a = "".join(a)
    a = re.findall(r"[\w']+", a)
    a = [ a[i].lower() for i in range(0,len(a)) ]
    a = [word for word in a if word not in stop_words]
    st = nltk.stem.RSLPStemmer()
    a = [ st.stem(a[i]) for i in range(0,len(a)) ]
    a = [x for x in a if x != "vid"]
    
    select = 0
    
    for element in a:
        if element in c10:
            select+=1
        else:
            pass
        if element in c12:
            select+=1
        else:
            pass
        if element in c14:
            select+=1
        else:
            pass
        if element in c16:
            select+=1
        else:
            pass
    return select